# Tiling

### Before the data can be used in the model, they have to tiled to 256x256


https://towardsdatascience.com/efficiently-splitting-an-image-into-tiles-in-python-using-numpy-d1bf0dd7b6f7

TODO
1) Tif lesen, georeferenzierten Punkt oben links für späteres Nutzen erstellen 
2) Georeferenzierung fallen lassen
3) tiling
4) Klassifizierung 
5) wieder zusammensetzten der Tiles 
6) Georeferenzierung wieder hinzufügen 

In [ ]:
import os
import sys
from PIL import Image

In [16]:
savedir = r"E:/W_Katrin/Kigali/Raster/tiles"
filename = r"E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif"

In [ ]:
img = Image.open(filename)
#tiling(filename, savedir, (256, 256))

In [18]:
from osgeo import gdal
dataset = gdal.Open(filename, gdal.GA_ReadOnly)
dataset

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000020F3BC19770> >

In [19]:
print(dataset.RasterCount)
  
# width 
print(dataset.RasterXSize)
  
# height
print(dataset.RasterYSize)

5
13162
8267


In [31]:
import cv2, os
base_path = "E:/W_Katrin/Kigali/Raster/"
new_path = "E:/W_Katrin/Kigali/Raster/"

#print(img)

im = gdal.Open(filename)
print(im)


outfile = filename.split('.')[0] + '.jpg'
print(outfile)
save(im, outputfile, "JPEG", quality=100)


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000020F3F0FA160> >
E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.jpg


NameError: name 'save' is not defined

In [41]:
import rasterio
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

raster = rasterio.open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif")
raster
band1 = raster.read(1)
band2 = raster.read(2)
band3 = raster.read(3)

rasterRGB = es.stack(band1, band2, band3)


ModuleNotFoundError: No module named 'earthpy'

In [42]:
out_meta = raster.meta.copy()
print(out_meta)
# save out_meta for later use!!

out_meta.update({"driver": "JPEG",
                 "transform": None,
                 "crs": None}
                         )
print(out_meta)

outraster = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.jpg"
with rasterio.open(outraster, "w", **out_meta) as dest:
       dest.write(raster)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 13162, 'height': 8267, 'count': 5, 'crs': CRS.from_epsg(32736), 'transform': Affine(0.4963054573117089, 0.0, 173723.36172185623,
       0.0, -0.4963054573117089, 9789230.701895246)}
{'driver': 'JPEG', 'dtype': 'float32', 'nodata': nan, 'width': 13162, 'height': 8267, 'count': 5, 'crs': None, 'transform': None}


CPLE_NotSupportedError: JPEG driver doesn't support 5 bands.  Must be 1 (grey), 3 (RGB) or 4 bands (CMYK). 

Thoughts: 
JPG can only hold 3 layers --- 
should maybe drop two layers anyways as was also trained to be 3 layers


In [12]:
# define tiling function 
def tiling(filename, savedir, target_size): 
    img = Image.open(filename)
    width, height = img.size
    start_pos = start_x, start_y = (0, 0)
    cropped_image_size = w, h = target_size
    frame_num = 1
    for col_i in range(0, width, w):
        for row_i in range(0, height, h):
            crop = img.crop((col_i, row_i, col_i + w, row_i + h))
            save_to= os.path.join(savedir, "testing_{row}_{col}.jpg".format(row=row_i, col=col_i))
            crop.save(save_to)


### Andere Methode, hier fehlen Randpieces! (nur 270 tiles, bei oberem mehr tiles, auch Randtiles)

### This method results in a list of arrays

## TODO

develop code to enter many images and return tiles in a logical way (also think of mask pixels!!) 

Problem: we have images, but we need the mask to be tiled at the same location the exact same way!